##### Once we get the simulations generated in a folder, each of the folders will contain trackrefiner folder under which we have the different simulations generated for different parameters. This code will copy the trackrefiner data into a new directory retaining its structure, and this will have our final data for node features. Additionally it also generates the object relationships file for every simulation.

In [ ]:
import torch.nn.functional as F
import os, shutil
import torch
import numpy as np
import pandas as pd
import gzip
from pathlib import Path
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch_geometric
from torch_geometric.data import Data, batch



In [98]:
directory = "/Users/sushmadhamodharan/Downloads/CSV_Files"

# Collect files starting with "Trackrefiner"
matching_files = []
for root, _, files in os.walk(directory):
    for file in files:
        if file.startswith("Trackrefiner"):
            matching_files.append(os.path.join(root, file))

destination_directory = Path("/Users/sushmadhamodharan/Downloads/finaldata")
if not destination_directory.exists():
    destination_directory.mkdir()
    
[(destination_directory / i).mkdir() for i in np.unique(list(map(lambda x: x[45:57], matching_files))) if not (destination_directory / i).exists()]
    
for file in matching_files:
    shutil.copy(file, destination_directory / file[45:57] / file.split("Trackrefiner/")[1])

In [119]:
#Generating parent child relationship
def generate_parentdf(df):
    parentdf = df[["stepNum", "ObjectNumber", "TrackObjects_ParentImageNumber_50", "TrackObjects_ParentObjectNumber_50"]].copy(deep=True)
    parentdf['Relationship'] = 'Parent'
    parentdf.columns = ["First Image Number", "First Object Number", "Second Image Number", "Second Object Number", "Relationship"]
    parentdf = parentdf[["Relationship", "First Image Number", "First Object Number", "Second Image Number", "Second Object Number"]].copy(deep=True)
    return parentdf

#Generating neighbor relationship
def compute_distance(position1, position2):
    
    distance = torch.norm(position1 - position2, dim=0)
    return distance

def generate_neighbor_df(df):
    distance_threshold = 1.0
    neighbordf = df[["stepNum", "ObjectNumber", "pos"]].copy(deep=True)
    edge_list = []
    for timestep in range(1, df.stepNum.max()):
        cells = neighbordf[neighbordf["stepNum"]== timestep].iloc[:, 1:].values

        for ix1, cell1 in enumerate(cells):
            for ix2, cell2 in enumerate(cells[(ix1+1):]):
                if compute_distance(cell1[1], cell2[1]) < distance_threshold:
                    edge_list.append(["Neighbors", timestep, cell1[0], timestep, cell2[0]])
                    
    ngdf = pd.DataFrame(edge_list, columns = ["Relationship", "First Image Number", "First Object Number", "Second Image Number", "Second Object Number"])
    return ngdf

def generate_fulldf(file):
    df = pd.read_csv(file)
    df['pos'] = df['pos'].apply(eval).apply(lambda x: torch.tensor(x, dtype = torch.float32))
    reldf = pd.concat([generate_parentdf(df), generate_neighbor_df(df)])
    return reldf
                      

In [120]:
matching_files = []
for root, _, files in os.walk(destination_directory):
    for file in files:
        if file.startswith("Trackrefiner"):
            matching_files.append(os.path.join(root, file))
            
for file in matching_files:
    generate_fulldf(file).to_csv(file.split("Trackrefiner")[0] + "ObjectRelationship.Trackrefiner" + file.split("Trackrefiner")[1])